In [1]:
import pandas as pd
import numpy as np

In [59]:
TRAINING_DATA_FILE = 'C:/Users/amar/ML Projects/SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'C:/Users/amar/ML Projects/SpamData/02_Training/test_data.txt'

TOKEN_SPAM_PROBE_FILE = 'C:/Users/amar/ML Projects/SpamData/03_Testing/prob_spam.txt'
TOKEN_HAM_PROB_FILE = 'C:/Users/amar/ML Projects/SpamData/03_Testing/prob_non_spam.txt'
TOKEN_ALL_PROB_FILE = 'C:/Users/amar/ML Projects/SpamData/03_Testing/prob_all_tokens.txt'

TEST_FEATURE_MATRIX = 'C:/Users/amar/ML Projects/SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'C:/Users/amar/ML Projects/SpamData/03_Testing/test-target.txt'


VOCAB_SIZE = 2500

### Read and Load Features from .txt Files into Numpy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter= ' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter= ' ', dtype= int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  1],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 14,  1,  1]])

In [6]:
sparse_train_data.shape

(292820, 4)

In [7]:
sparse_test_data[:5]

array([[8, 2, 1, 1],
       [8, 3, 1, 2],
       [8, 4, 1, 4],
       [8, 5, 1, 2],
       [8, 6, 1, 1]])

In [8]:
sparse_test_data.shape

(128813, 4)

In [9]:
print('num of rows in trainig file', sparse_train_data.shape[0])
print('num of rows in testing file', sparse_test_data.shape[0])

num of rows in trainig file 292820
num of rows in testing file 128813


In [21]:
print('num of email in training file', np.unique(sparse_train_data[:, 0]).size)

num of email in training file 4782


In [12]:
print('num of email in test file', np.unique(sparse_test_data[:, 0]).size)

num of email in test file 2052


### sparse to full matrix DataFrame

In [18]:
#create Empty Dataframe
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
index_names = np.unique(sparse_train_data[:, 0])
full_train_data = pd.DataFrame(index=index_names, columns=column_names)  #empty df
full_train_data.fillna(value= 0, inplace=True)  #now filled with zero
full_train_data


,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6893,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6894,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
## Create full Matrix from sparse matrix
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix


In [22]:
%%time 

full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 21 s


In [23]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,1,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,4,2,3,0,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6888,0,1,2,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6892,0,3,2,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6893,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
%%time 

full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 7.6 s


In [25]:
full_test_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,2,4,2,1,4,1,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,1,7,1,2,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6885,0,2,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6887,0,2,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6889,0,2,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### TOKEN PROBABILITIES
### count the token to train naive bayes model

In [26]:
#CALCULATE PROBABILITY OF SPAM
full_train_data.CATEGORY.size   #total num of emails

4782

In [27]:
full_train_data.CATEGORY.sum()   # nom of spam emails(1+1...)

1247

In [28]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print('Probability of spam is :', prob_spam)

Probability of spam is : 0.26076955248849853


In [29]:
# total numbers of words/tokes
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,1,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,4,2,3,0,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [30]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4782,)

In [31]:
email_lengths.head()

DOC_ID
0     86
1     52
2     39
3    181
4     43
dtype: int64

In [32]:
total_wc = email_lengths.sum()   #total num of words
total_wc

479126

### number of tokens/words in Spam and Ham Emails

In [35]:
 #spam message and number
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]  
spam_lengths.shape

(1247,)

In [36]:
 #sum of all the word in spam messages
spam_wc = spam_lengths.sum()          
spam_wc

178969

In [37]:
 #ham message and numbers
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]   
ham_lengths.shape

(3535,)

In [38]:
 #sum of all the word in ham messages
ham_wc = ham_lengths.sum()             
ham_wc

300157

In [39]:
ham_wc + spam_wc - total_wc

0

In [42]:
print('Avaerage num of words in spam messages {:.0f} '.format(spam_wc/spam_lengths.shape[0]))
print('Avaerage num of words in ham messages {:.0f} '.format(ham_wc/spam_lengths.shape[0]))

Avaerage num of words in spam messages 144 
Avaerage num of words in ham messages 241 


### summing tokens occuring in spam

In [43]:
full_train_features.shape


(4782, 2500)

In [44]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,1,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,4,2,3,0,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [45]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1892,2,0,2,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [46]:
train_spam_tokens.shape

(1247, 2500)

In [47]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) +1  #(laplas smoothing adding 1)
summed_spam_tokens.shape

(2500,)

### summing tokens occuring in ham

In [48]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,1,0,2,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
1898,2,1,0,1,1,2,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
train_ham_tokens.shape

(3535, 2500)

In [50]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) +1  #(laplas smoothing adding 1)
summed_ham_tokens.shape

(2500,)

### p(Token|spam ) - Probability that a token occures given the email is spam

In [51]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012019
1    0.005202
2    0.007335
3    0.007153
4    0.010729
dtype: float64

In [52]:
prob_tokens_spam.sum()

1.0

### p(Token|ham ) - Probability that a token occures given the email is ham

In [54]:
prob_tokens_ham = summed_ham_tokens / (ham_wc + VOCAB_SIZE)   #for laplas smoothing we add VOCAB_SIZE in denominator
prob_tokens_ham[:5]

0    0.020204
1    0.010094
2    0.006393
3    0.006231
4    0.003149
dtype: float64

In [55]:
prob_tokens_ham.sum()

1.0

In [56]:
prob_token_all = full_train_features.sum(axis= 0) / total_wc

In [57]:
prob_token_all.sum()

1.0

### save the trained model

In [60]:
np.savetxt(TOKEN_SPAM_PROBE_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_token_all)

In [61]:
# Prepare test data
full_test_data


,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,2,4,2,1,4,1,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,1,7,1,2,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6885,0,2,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6887,0,2,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6889,0,2,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [65]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)